In [ ]:
import tensorflow as tf
num_gpus = 1
l = tf.reshape(tf.tile(tf.range(100, dtype=tf.float32), [64]), [64, 100])
l_by_gpu = tf.split(l, num_gpus)
sums_by_gpu = [tf.zeros([]) for _ in range(num_gpus)]

def body(sums_by_gpu, l_by_gpu, ref=None):
    new_sum, new_l = [], []
    for gpu_idx, (sum_, l) in enumerate(zip(ref[0], ref[1])):
#         with tf.device('/gpu:{}'.format(gpu_idx)):
#             sum_ += tf.reduce_sum(l[:, 0])
#             l = tf.split(l, tf.stack([1, tf.shape(l)[-1]-1]), axis=1)[1]
#             l = tf.Print(l, [tf.zeros([])])
#             new_sum.append(sum_)
#             new_l.append(l)
        sum_ += tf.reduce_sum(l[:, 0])
        l = tf.split(l, tf.stack([1, tf.shape(l)[-1]-1]), axis=1)[1]
        l = tf.Print(l, [tf.zeros([])])
        new_sum.append(sum_)
        new_l.append(l)
    return new_sum, new_l

def cond(sums_by_gpu, l_by_gpu):
    return tf.cast(tf.shape(l_by_gpu[0])[1], tf.bool)

print(id(sums_by_gpu))
sums, _ = tf.while_loop(
    cond,
    lambda x, y: body(x, y, ref=[sums_by_gpu, l_by_gpu]),
    [sums_by_gpu, l_by_gpu],
    shape_invariants=[
        [sums_by_gpu[0].get_shape()]*len(sums_by_gpu),
        [tf.TensorShape([None, None])]*len(l_by_gpu)
    ]
)
print(id(sums))
gpu_options = tf.GPUOptions(allow_growth=True)
with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) as sess:
    sums = sess.run(sums)
    print(sums)

140044272168136
140044271616136


In [ ]:
from deeppavlov.core.commands.train import build_model_from_config
from deeppavlov.core.data.dataset_reader import DatasetReader
#from deeppavlov.core.data.utils import download_decompress
from deeppavlov.core.common.registry import register

In [28]:
import timeit

print(
    timeit.timeit(
        stmt='s.extend([5, 6, 7, 8, 9, 10]);print(s)',
        setup='s = [1, 2, 3, 4]',
        number=10,
    )
)
print(
    timeit.timeit(
        stmt='s += [5, 6, 7, 8, 9, 10]',
        setup='s = [1, 2, 3, 4]',
        number=10,
    )
)
print(
    timeit.timeit(
        stmt='s = s + [5, 6, 7, 8, 9, 10]',
        setup='s = [1, 2, 3, 4]',
        number=10,
    )
)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 5, 6, 7, 8, 9, 10]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 5, 6, 7, 8, 9, 10, 5, 6, 7, 8, 9, 10]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 5, 6, 7, 8, 9, 10, 5, 6, 7, 8, 9, 10, 5, 6, 7, 8, 9, 10]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 5, 6, 7, 8, 9, 10, 5, 6, 7, 8, 9, 10, 5, 6, 7, 8, 9, 10, 5, 6, 7, 8, 9, 10]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 5, 6, 7, 8, 9, 10, 5, 6, 7, 8, 9, 10, 5, 6, 7, 8, 9, 10, 5, 6, 7, 8, 9, 10, 5, 6, 7, 8, 9, 10]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 5, 6, 7, 8, 9, 10, 5, 6, 7, 8, 9, 10, 5, 6, 7, 8, 9, 10, 5, 6, 7, 8, 9, 10, 5, 6, 7, 8, 9, 10, 5, 6, 7, 8, 9, 10]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 5, 6, 7, 8, 9, 10, 5, 6, 7, 8, 9, 10, 5, 6, 7, 8, 9, 10, 5, 6, 7, 8, 9, 10, 5, 6, 7, 8, 9, 10, 5, 6, 7, 8, 9, 10, 5, 6, 7, 8, 9, 10]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 5, 6, 7, 8, 9, 10, 5, 6, 7, 8, 9, 10, 5, 6, 7, 8, 9, 10, 5, 6, 7, 8, 9, 10, 5, 6, 7, 8, 9, 10, 5, 6, 7, 8, 9, 10, 5, 6, 7, 8, 9, 10, 5, 6, 7, 8, 9, 10]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 

In [16]:
import numpy as np

def process_1_squash(borders, i):
    print(borders, i)
#     left, right = borders[0], borders[1]
#     left_idx, right_idx = round(left), np.ceil(right)
#     num_ones = right_idx - left_idx - 1
#     if left == left_idx:
#         num_ones += 1
#     if right == right_idx:
#         num_ones += 1
#     indices = list(zip(range(left_idx, right_idx+1), [i]*(right_idx - left_idx + 1)))
#     values = [np.ceil(left) - left] if left_idx < left else []
#     values += [1] * num_ones
#     if right != right_idx:
#         values.append(right - round(right))
#     return indices, values

def squash_sparse_matrix(N, M):
    factor = N / M
    borders = np.stack([np.arange(N, 0, -1), np.arange(1, N+1)]) * factor
    # print(borders.shape)
    borders_sep = np.split(borders, borders.shape[0])
    # print(borders_sep)
    vfunc = np.vectorize(process_1_squash, otypes=[np.int32, np.float32])
    return vfunc(borders, tuple(range(N)))
    
print(squash_sparse_matrix(10, 5))

20.0 0
(array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32), array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32))


In [4]:
text = "Напишите программу вычисляющую сколько раз каждое слово \
встречается в поданной на вход тексте строке Результаты подсчета \
поместите в словарь а затем распечатайте словарь в порядке убывания \
встречаемости слов В каждой строке сначала печатается номер места \
по встречаемости которое заняло слово а затем через пробел то \
сколько раз оно встретилось и само слово  Гарантируется что во \
входном тексте не будет символов кроме букв и пробелов Перед \
сортировкой весь текст следует перевести в нижний регистр с помощью метода strlower"
m = text.lower().split()
t = []
d = {}
for i in m:
    t = m.count(i)
    d[i] = t
d1 = sorted(d.items(), key=lambda x: x[1])
#print(t)
print(d1)

[('напишите', 1), ('программу', 1), ('вычисляющую', 1), ('каждое', 1), ('встречается', 1), ('поданной', 1), ('на', 1), ('вход', 1), ('результаты', 1), ('подсчета', 1), ('поместите', 1), ('распечатайте', 1), ('порядке', 1), ('убывания', 1), ('слов', 1), ('каждой', 1), ('сначала', 1), ('печатается', 1), ('номер', 1), ('места', 1), ('по', 1), ('которое', 1), ('заняло', 1), ('через', 1), ('пробел', 1), ('то', 1), ('оно', 1), ('встретилось', 1), ('само', 1), ('гарантируется', 1), ('что', 1), ('во', 1), ('входном', 1), ('не', 1), ('будет', 1), ('символов', 1), ('кроме', 1), ('букв', 1), ('пробелов', 1), ('перед', 1), ('сортировкой', 1), ('весь', 1), ('текст', 1), ('следует', 1), ('перевести', 1), ('нижний', 1), ('регистр', 1), ('с', 1), ('помощью', 1), ('метода', 1), ('strlower', 1), ('сколько', 2), ('раз', 2), ('тексте', 2), ('строке', 2), ('словарь', 2), ('а', 2), ('затем', 2), ('встречаемости', 2), ('и', 2), ('слово', 3), ('в', 5)]


In [2]:
import numpy as np
def f(x, y):
    return max(x, y)

vecf = np.vectorize(f)

print(vecf([i for i in range(10)], [i+1 for i in range(10)]))

[ 1  2  3  4  5  6  7  8  9 10]


In [1]:
import tensorflow as tf

distr = [0.05, 0.30, 0.60, 0.05]
N = len(distr)
batch_size = 31

cum_distr = distr[:1]
for p in distr[1:]:
    cum_distr.append(cum_distr[-1] + p)
    
cum_distr = tf.constant(cum_distr)

x = tf.Variable(tf.truncated_normal([N]))

unif = tf.random_uniform([batch_size, 1])
comp = tf.to_float(cum_distr > unif)
all_labels = tf.eye(N)
indices = tf.argmax(comp, output_type=tf.int32, axis=-1)
labels = tf.gather(all_labels, indices)
probs = tf.nn.softmax(x)

loss = tf.nn.softmax_cross_entropy_with_logits_v2(logits=tf.stack([x]*batch_size), labels=labels)

global_step = tf.Variable(0, trainable=False)
learning_rate = tf.train.exponential_decay(0.001, global_step, 1000, 0.5, staircase=True)
opt = tf.train.AdamOptimizer(learning_rate)
train_op = opt.minimize(loss)

In [2]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(100000):
        _, pr = sess.run([train_op, probs])
#         print(cd)
#         print(u)
#         print(c)
        if i % 1000 == 0:
            print(i, pr)


0 [0.08096984 0.0975479  0.26694262 0.5545396 ]
1000 [0.0533044  0.19900034 0.5111163  0.23657893]
2000 [0.04506009 0.2585933  0.5747717  0.12157496]
3000 [0.0462773  0.28025383 0.5934374  0.08003151]
4000 [0.04730603 0.2972135  0.59320414 0.06227635]
5000 [0.04869702 0.29630738 0.6007808  0.05421491]
6000 [0.04989836 0.302922   0.5937062  0.05347345]
7000 [0.04912981 0.30450043 0.59776264 0.04860713]
8000 [0.05085437 0.301653   0.5983184  0.04917423]
9000 [0.04980492 0.30537108 0.59524184 0.0495821 ]
10000 [0.04901305 0.29486674 0.60645235 0.0496678 ]
11000 [0.05039519 0.30151966 0.5997605  0.04832472]
12000 [0.05130767 0.2991871  0.6008251  0.04868015]
13000 [0.04810471 0.30305338 0.59956044 0.04928147]
14000 [0.05046886 0.3014141  0.59800047 0.05011658]
15000 [0.05116741 0.2969777  0.6018445  0.0500104 ]
16000 [0.05030151 0.30790877 0.5929822  0.04880754]
17000 [0.05044535 0.30435082 0.59567946 0.04952439]
18000 [0.05145176 0.30772248 0.59086555 0.04996015]
19000 [0.04869069 0.29502

In [20]:

def indices_and_weights_for_squeezing_of_1_neuron(i, N, M):
    weights = []
    indices = []
    q = N / M
    left, right = i * q, (i + 1) * q
    if int(left) == left:
        start_of_ones = int(left)
    else:
        weights.append(np.ceil(left) - left)
        indices.append(int(left))
        start_of_ones = int(left) + 1
    
    end_of_ones = int(right)
    weights += [1.] * (end_of_ones - start_of_ones)
    indices.extend(range(start_of_ones, end_of_ones))
    
    if int(right) != right:
        weights.append(right - int(right))
        indices.append(indices[-1] + 1)
        
    factor = q ** -0.5
    weights = [w * factor for w in weights]
    return indices, weights

N = 10
M = 3
for i in range(M):
    print(indices_and_weights_for_squeezing_of_1_neuron(i, N, M))

([0, 1, 2, 3], [0.5477225575051661, 0.5477225575051661, 0.5477225575051661, 0.18257418583505544])
([3, 4, 5, 6], [0.36514837167011066, 0.5477225575051661, 0.5477225575051661, 0.3651483716701109])
([6, 7, 8, 9], [0.1825741858350552, 0.5477225575051661, 0.5477225575051661, 0.5477225575051661])


In [21]:
def squeezing_sparse_matrix(N, M):
    init_indices, init_weights = [], []
    for i in range(M):
        indices, weights = indices_and_weights_for_squeezing_of_1_neuron(i, N, M)
        init_indices.extend(zip(indices, [i] * len(indices)))
        init_weights += weights
    return init_indices, init_weights

print(init_of_squeezing_sparse_matrix(N, M))

([(0, 0), (1, 0), (2, 0), (3, 0), (3, 1), (4, 1), (5, 1), (6, 1), (6, 2), (7, 2), (8, 2), (9, 2)], [0.5477225575051661, 0.5477225575051661, 0.5477225575051661, 0.18257418583505544, 0.36514837167011066, 0.5477225575051661, 0.5477225575051661, 0.3651483716701109, 0.1825741858350552, 0.5477225575051661, 0.5477225575051661, 0.5477225575051661])


In [30]:
import tensorflow as tf

v = tf.placeholder(tf.float32, shape=[1, None])
r = tf.concat([v, tf.zeros([1, 2])], 0)
with tf.Session() as sess:
    print(sess.run(r, feed_dict={v: [[1, 2]]}))

[[1. 2.]
 [0. 0.]]


In [7]:
from deeppavlov.models.tokenizers.ru_tokenizer import RussianTokenizer

tokenizer = RussianTokenizer(ngram_range=[1, 1])
batch = ['abc', 'def']
print(tokenizer(batch))

2018-08-01 15:29:52.751 INFO in 'pymorphy2.opencorpora_dict.wrapper'['wrapper'] at line 16: Loading dictionaries from /home/anton/dpenv/lib/python3.6/site-packages/pymorphy2_dicts/data
2018-08-01 15:29:52.805 INFO in 'pymorphy2.opencorpora_dict.wrapper'['wrapper'] at line 20: format: 2.4, revision: 393442, updated: 2015-01-17T16:03:56.586168


[['abc'], ['def']]


In [15]:
import collections

d = collections.OrderedDict(a=1, b=2)
print(type(d) == collections.OrderedDict)

tuple_type = collections.namedtuple('mytuple', [])

t = tuple_type()
print(tuple_type.__bases__)
print(getattr(tuple_type, '_fields'))
print(all([]))

True
(<class 'tuple'>,)
()
True


In [1]:
import os
import tensorflow as tf
from tensorflow.contrib.cudnn_rnn import CudnnLSTM as CudnnLSTM
inp = tf.zeros([10, 32, 100])
lstm1 = CudnnLSTM(1, 128)
lstm2 = CudnnLSTM(2, 256)
with tf.name_scope('rnn1'):
    lstm1.build(inp.shape)
with tf.name_scope('rnn2'):
    lstm2.build(inp.shape)
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    save_path = os.path.join('test_cudnn_lstm_save', '1')
    if not os.path.exists(save_path):
        os.makedirs(os.path.join(save_path))
    saver.save(sess, save_path)

InvalidArgumentError: Adding duplicate key: rnn/multi_rnn_cell/cell_0/cudnn_compatible_lstm_cell/kernel
	 [[Node: save/SaveV2 = SaveV2[dtypes=[DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save/Const_0_0, save/SaveV2/tensor_names, save/SaveV2/shape_and_slices, cudnn_lstm/opaque_kernel/_1, rnn1/transpose/_3, rnn1/concat_5/_5, cudnn_lstm_1/opaque_kernel/_7, rnn2/transpose/_9, rnn2/transpose_1/_11, rnn2/concat_5/_13, rnn2/concat_11/_15)]]

Caused by op 'save/SaveV2', defined at:
  File "/usr/local/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_ins

In [14]:
import numpy as np
from scipy.sparse import coo_matrix

def indices_and_weights_for_squeezing_of_1_neuron(i, N, M):
    weights = []
    indices = []
    q = N / M
    left, right = i * q, (i + 1) * q
    if N >= M:
        if int(left) == left:
            start_of_ones = int(left)
        else:
            weights.append(np.ceil(left) - left)
            indices.append(int(left))
            start_of_ones = int(left) + 1

        end_of_ones = int(right)
        weights += [1.] * (end_of_ones - start_of_ones)
        indices.extend(range(start_of_ones, end_of_ones))

        if int(right) != right:
            weights.append(right - int(right))
            indices.append(indices[-1] + 1)
    else:
        left_floor, right_floor = int(left), int(right)
        if left_floor == right_floor or right == right_floor:
            weights.append(right - left)
            indices.append(left_floor)
        else:
            weights += [right_floor - left, right - right_floor]
            indices += [left_floor, right_floor]

    factor = sum([w**2 for w in weights]) ** -0.5
    weights = [w * factor for w in weights]
    return indices, weights

def squeezing_sparse_matrix(N, M):
    init_indices, init_weights = [], []
    for i in range(M):
        # print("(util.tensor.squeezing_sparse_matrix)N, M:", N, M)
        indices, weights = indices_and_weights_for_squeezing_of_1_neuron(i, N, M)
        init_indices.extend(zip(indices, [i] * len(indices)))
        init_weights += weights
    return init_indices, init_weights

N, M = 5, 2
indices, weights = squeezing_sparse_matrix(N, M)
row_ids, col_ids = zip(*indices)
print(row_ids, col_ids)
sp_matrix = coo_matrix((weights, (row_ids, col_ids)), shape=(N, M))
print(sp_matrix.todense())



(0, 1, 2, 2, 3, 4) (0, 0, 0, 1, 1, 1)
[[0.66666667 0.        ]
 [0.66666667 0.        ]
 [0.33333333 0.33333333]
 [0.         0.66666667]
 [0.         0.66666667]]


In [28]:
x = input("Do you need the answer? (y/n): ")
if x.lower() == "y":
    required = True
else:
    required = False
    
def the_answer(self, *args):              
        return 42
    
class EssentialAnswers(type):
    
    def __init__(cls, clsname, superclasses, attributedict):
        if required:
            cls.the_answer = the_answer
                           
    
class Philosopher1(metaclass=EssentialAnswers): 
    pass


print(Philosopher1.the_answer)
class Philosopher2(metaclass=EssentialAnswers): 
    pass
class Philosopher3(metaclass=EssentialAnswers): 
    pass
    
    
plato = Philosopher1()
print(plato.the_answer())
kant = Philosopher2()
# let's see what Kant has to say :-)
print(kant.the_answer())

Do you need the answer? (y/n): y


AttributeError: type object 'EssentialAnswers' has no attribute 'the_answer'

In [29]:
from char_lm_vocab import CharLMVocabulary
vocab = CharLMVocabulary(
    save_path='./vocab.dict',
    load_path='./vocab.dict',
)
text = 'DeepPavlov is an open-source conversational AI library built on TensorFlow and Keras.'
vocab.fit(text)
vocab.save()

2018-08-03 15:36:34.385 INFO in 'char_lm_vocab'['char_lm_vocab'] at line 70: [loading vocabulary from /home/anton/DeepPavlov/download/vocab.dict]
2018-08-03 15:36:34.400 INFO in 'char_lm_vocab'['char_lm_vocab'] at line 70: [loading vocabulary from /home/anton/DeepPavlov/download/vocab.dict]
2018-08-03 15:36:34.412 INFO in 'char_lm_vocab'['char_lm_vocab'] at line 59: [saving vocabulary to /home/anton/DeepPavlov/download/vocab.dict]


In [31]:
print(len(vocab))
print(vocab[5])
print('z' in vocab)
print('a' in vocab)
print(vocab.is_str_batch([]))
print(vocab.is_str_batch(['c']))
print(vocab(['a', 'b', 'я']))

27
r
False
True
False
True
[3, 14, None]


In [1]:
import tensorflow as tf
from tensorflow.contrib.cudnn_rnn import CudnnLSTM as LSTM
import numpy as np

num_units = 100
voc_size = 20
inps = tf.placeholder(tf.int32, shape=[None, None])
lbls = tf.placeholder(tf.int32, shape=[None, None])
prep_inps = tf.one_hot(inps, voc_size)
prep_lbls = tf.one_hot(lbls, voc_size)

out_w = tf.Variable(
    tf.truncated_normal([num_units, voc_size])
)

with tf.device('/gpu:0'):
    lstm = LSTM(
        2,
        num_units,
        input_mode='skip_input',
    )

bs = tf.shape(inps)[1:-1]
state_shape = tf.concat([[2], bs, [num_units]], 0)
with tf.device('/gpu:0'):
    lstm_res, state = lstm(
        prep_inps
    )
    print(lstm_res)
    logits = tf.matmul(tf.reshape(lstm_res, [-1, num_units]), out_w)
    predictions = tf.nn.softmax(logits)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
        logits=logits,
        labels=tf.reshape(prep_lbls, [-1, voc_size])
    ))
    opt = tf.train.GradientDescentOptimizer(1.)
    train_op = opt.minimize(loss)
    
nu = 10
bs = 32
    
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for _ in range(100):
        inputs = np.random.randint(0, voc_size, size=(nu, bs))
        labels = np.random.randint(0, voc_size, size=(nu, bs))
        _, preds = sess.run([train_op, preds], feed_dict={inps: inputs, lbls: labels})

KeyboardInterrupt: 

In [3]:
import itertools

def create_distribute_map(num_distributed, result_length):
    quotient = result_length / num_distributed
    num_filled = 0
    if num_distributed < result_length:
        num_repeats = list()
        for i in range(1, num_distributed):
            filled_space = quotient * i
            print(filled_space)
            truncated = int(filled_space)
            num_repeats.append(truncated - num_filled)
            num_filled = truncated
        num_repeats.append(result_length - num_filled)
    else:
        num_repeats = [1] * result_length
    map_ = list(itertools.chain(*[[i] * n_rep for i, n_rep in enumerate(num_repeats)]))
    return map_

print(create_distribute_map(3, 2))

[0, 1]


In [5]:
def f():
    pass
h = f
g = lambda x: x
i = lambda x: x
print(h)
print(g)
print(i)
print(f)

<function f at 0x7ff2b407ac80>
<function <lambda> at 0x7ff2b407ad90>
<function <lambda> at 0x7ff2b407a9d8>
<function f at 0x7ff2b407ac80>


In [9]:
import re
text = "Mama и Папа"
text = re.sub("([A-ZА-Я])", r"^\1", text)
print(text)

^Mama и ^Папа


In [13]:
from helmo.util.text import preprocessing, postprocessing
text = "A component of Tool's song repertoire relies on the use of unusual time signatures. For instance, Chancellor describes the time signature employed on the first single from Lateralus, \"Schism\", as \"six\" and \"six-and-a-half\" and that it later \"goes into all kinds of other times\".[115] Further examples include the album's title track, which also displays shifting rhythms,[115] as do 10,000 Days: \"Wings for Marie (Pt 1)\" and \"10,000 Days (Wings Pt 2)\".[116]"
text = preprocessing.hat_uppercase(text)
print(text)
print()
text = postprocessing.hat_uppercase(text)
print(text)

^a component of ^tool's song repertoire relies on the use of unusual time signatures. ^for instance, ^chancellor describes the time signature employed on the first single from ^lateralus, "^schism", as "six" and "six-and-a-half" and that it later "goes into all kinds of other times".[115] ^further examples include the album's title track, which also displays shifting rhythms,[115] as do 10,000 ^days: "^wings for ^marie (^pt 1)" and "10,000 ^days (^wings ^pt 2)".[116]

A component of Tool's song repertoire relies on the use of unusual time signatures. For instance, Chancellor describes the time signature employed on the first single from Lateralus, "Schism", as "six" and "six-and-a-half" and that it later "goes into all kinds of other times".[115] Further examples include the album's title track, which also displays shifting rhythms,[115] as do 10,000 Days: "Wings for Marie (Pt 1)" and "10,000 Days (Wings Pt 2)".[116]


In [16]:
import os

def load_tt_results(result_dir, required_metric_names):
    folders = filter(lambda x: x.isdigit(), os.listdir(result_dir))
    metrics = list()
    launches_for_testing = list()
    trained_launches = list()
    for folder in folders:
        one_launch_metrics = dict()
        path_to_metrics = os.path.join(result_dir, folder, 'testing/results')
        if os.path.exists(path_to_metrics):
            for metric_file in os.listdir(path_to_metrics):
                metric_name = metric_file.split('_')[0]
                with open(os.path.join(path_to_metrics, metric_file), 'r') as f:
                    one_launch_metrics[metric_name] = float(f.read())
        if set(required_metric_names) == set(one_launch_metrics.keys()):
            metrics.append(one_launch_metrics)
            trained_launches.append(folder)
        elif len(one_launch_metrics) > 0:
            launches_for_testing.append(folder)
            trained_launches.append(folder)
    return metrics, launches_for_testing, trained_launches


required_metric_names = ['accuracy', 'bpc', 'perplexity', 'loss']
result_dir = 'expres/resrnn/char/tt/first'
print(load_tt_results(result_dir, required_metric_names))

([{'loss': 0.8077297808527947, 'bpc': 1.2, 'perplexity': 4.011995053052902, 'accuracy': 0.7565445318222046}], [], ['0'])


In [5]:
L1 = [1, 2, 3, 4]
print("L1:", L1)
L2 = [10, 11, 12]
L1[:2] = L2
print("\nL2:", L2)
print("L1 after 'L1[:2] = L2':", L1)
L3 = [-3, -4]
L1[1:2] = L3
print("\nL3:", L3)
print("L1 after 'L1[1:2] = L3':", L1)
L4 = [5, 6, 7, 8]
L1[3:3] = L4
print("\nL4:", L4)
print("L1 after 'L1[3:3] = L4':", L1)

L1: [1, 2, 3, 4]

L2: [10, 11, 12]
L1 after 'L1[:2] = L2': [10, 11, 12, 3, 4]

L3: [-3, -4]
L1 after 'L1[1:2] = L3': [10, -3, -4, 12, 3, 4]

L4: [5, 6, 7, 8]
L1 after 'L1[3:3] = L4': [10, -3, -4, 5, 6, 7, 8, 12, 3, 4]


ValueError: attempt to assign sequence of size 2 to extended slice of size 1

In [7]:
import itertools
import copy
from useful_functions import all_combs

def create_even_distribution(num_unique_items, num_slots):
    """Returns a list with number of slots taken by each item. If num_slots < num_unique_items not all items
    will be distributed"""
    items_per_slot_float = num_slots / num_unique_items
    total_slots_filled_saved = 0
    if num_unique_items < num_slots:
        item_distribution = list()
        for i in range(1, num_unique_items):
            total_slots_float = items_per_slot_float * i
            total_slots_filled = int(total_slots_float)
            item_distribution.append(total_slots_filled - total_slots_filled_saved)
            total_slots_filled_saved = total_slots_filled
        item_distribution.append(num_slots - total_slots_filled_saved)
    else:
        item_distribution = [1] * num_slots
    return item_distribution


def create_distribute_map(num_unique_items, num_slots):
    item_distribution = create_even_distribution(num_unique_items, num_slots)
    map_ = list(itertools.chain(*[[i] * n_rep for i, n_rep in enumerate(item_distribution)]))
    return map_


def approx_mem_consumption(config):
    # in MB for sequence_length=100, batch_size=128, num_layers=2, num_units=2000
    # working for all lstm variants
    base_consumption = 4500
    consumption = base_consumption * \
        (config['num_units'] / 2000)**2 * \
        (config['num_layers'] / 2) * \
        (config['batch_size'] / 128) * \
        (config['sequence_length'] / 100)
    return consumption


def num_consequent_repeats(list_, idx):
    current = list_[idx]
    i = 1
    while list_[idx+i] == current:
        i += 1
    return i


def ceil(a):
    if int(a) != a:
        return int(a) + 1
    return int(a)


def split_experiment_config_into_separate_measurement_configs(config):
    seq_lens = make_list(config['sequence_length'])
    batch_sizes = make_list(config['batch_size'])
    num_layers = make_list(config['num_layers'])
    num_units = make_list(config['num_units'])
    combs = all_combs([seq_lens, batch_sizes, num_layers, num_units])
    configs = list()
    for comb in combs:
        conf = copy.deepcopy(config)
        conf['sequence_length'] = comb[0]
        conf['batch_size'] = comb[1]
        conf['num_layers'] = comb[2]
        conf['num_units'] = comb[3]
        configs.append(conf)
    if config['num_repeats'] == 1:
        return configs
    else:
        confs = list()
        for conf in configs:
            confs.extend([conf]*config['num_repeats'])
        return confs


def get_configs_run_in_parallel(configs, counter):
    current_config = configs[counter]
    max_num_in_parallel = current_config['memory_per_experiment'] / approx_mem_consumption(current_config)
    num_repeats = num_consequent_repeats(configs, counter)
    configs_to_process = configs[counter:counter+num_repeats]
    num_launches_required = ceil(num_repeats / max_num_in_parallel)
    distribution = create_even_distribution(num_launches_required, num_repeats)
    list_of_launches = list()
    pointer = 0
    for num_proc in distribution:
        list_of_launches.append(configs_to_process[pointer:pointer+num_proc])
        pointer += num_proc
    return list_of_launches, num_repeats


def make_list(candidate):
    if isinstance(candidate, list):
        l = candidate
    else:
        l = [candidate]
    return l

In [16]:
config = {
  "num_layers": [3, 4],
  "num_units": [1000, 2000],
  "sequence_length": [50, 100],
  "batch_size": [128],
  "save_path": "results/lstm_test",
  "lstm_type": "cudnn_stacked",
  "mode": "train",
  "num_steps": 500,
  "num_proc_in_pool": 5,
  "num_repeats": 10,
  "memory_per_experiment": 7500
}

In [17]:
configs = split_experiment_config_into_separate_measurement_configs(config)
launches = get_configs_run_in_parallel(configs, 0)
for idx, launch in enumerate(launches[0]):
    print(idx)
    for conf in launch:
        print(conf)
    print('\n')

0
{'num_units': 1000, 'num_steps': 500, 'num_layers': 3, 'memory_per_experiment': 7500, 'mode': 'train', 'sequence_length': 50, 'save_path': 'results/lstm_test', 'batch_size': 128, 'num_proc_in_pool': 5, 'lstm_type': 'cudnn_stacked', 'num_repeats': 10}
{'num_units': 1000, 'num_steps': 500, 'num_layers': 3, 'memory_per_experiment': 7500, 'mode': 'train', 'sequence_length': 50, 'save_path': 'results/lstm_test', 'batch_size': 128, 'num_proc_in_pool': 5, 'lstm_type': 'cudnn_stacked', 'num_repeats': 10}
{'num_units': 1000, 'num_steps': 500, 'num_layers': 3, 'memory_per_experiment': 7500, 'mode': 'train', 'sequence_length': 50, 'save_path': 'results/lstm_test', 'batch_size': 128, 'num_proc_in_pool': 5, 'lstm_type': 'cudnn_stacked', 'num_repeats': 10}
{'num_units': 1000, 'num_steps': 500, 'num_layers': 3, 'memory_per_experiment': 7500, 'mode': 'train', 'sequence_length': 50, 'save_path': 'results/lstm_test', 'batch_size': 128, 'num_proc_in_pool': 5, 'lstm_type': 'cudnn_stacked', 'num_repeats'

In [18]:
print(approx_mem_consumption(configs[0]))

843.75


In [1]:
for i in range(20):
    print(2**(2*i) - 2**i + 2)

2
4
14
58
242
994
4034
16258
65282
261634
1047554
4192258
16773122
67100674
268419074
1073709058
4294901762
17179738114
68719214594
274877382658


In [2]:
for i in range(20):
    print(i**2 - i + 2)

2
2
4
8
14
22
32
44
58
74
92
112
134
158
184
212
242
274
308
344


In [3]:
import tensorflow as tf
from tensorflow.nn.rnn_cell import LSTMCell as LSTMCell
lstms = [LSTMCell(num_units, dtype=tf.float32) for num_units in [100, 200, 300]]
multilayer_lstm = tf.contrib.rnn.MultiRNNCell(lstms)
print(multilayer_lstm.zero_state(10, tf.float32))

(LSTMStateTuple(c=<tf.Tensor 'MultiRNNCellZeroState_1/LSTMCellZeroState/zeros:0' shape=(10, 100) dtype=float32>, h=<tf.Tensor 'MultiRNNCellZeroState_1/LSTMCellZeroState/zeros_1:0' shape=(10, 100) dtype=float32>), LSTMStateTuple(c=<tf.Tensor 'MultiRNNCellZeroState_1/LSTMCellZeroState_1/zeros:0' shape=(10, 200) dtype=float32>, h=<tf.Tensor 'MultiRNNCellZeroState_1/LSTMCellZeroState_1/zeros_1:0' shape=(10, 200) dtype=float32>), LSTMStateTuple(c=<tf.Tensor 'MultiRNNCellZeroState_1/LSTMCellZeroState_2/zeros:0' shape=(10, 300) dtype=float32>, h=<tf.Tensor 'MultiRNNCellZeroState_1/LSTMCellZeroState_2/zeros_1:0' shape=(10, 300) dtype=float32>))


In [4]:
print(tf.__version__)

1.10.0


In [6]:
a = tf.Variable(0, trainable=False, validate_shape=False)

In [7]:
b = ([1], [2], [3])
for k in zip(*b):
    print(k)

(1, 2, 3)


In [13]:
def deep_zip(objects, depth, passable_types=(list, tuple, dict)):
    if depth != 0 and isinstance(objects[0], passable_types):
        if isinstance(objects[0], (list, tuple)):
            zipped = list()
            for comb in zip(*objects):
                zipped.append(
                    deep_zip(comb, depth-1, passable_types=passable_types)
                )
            if isinstance(objects[0], tuple):
                zipped = tuple(zipped)
            return zipped
        elif isinstance(objects[0], dict):
            zipped = dict()
            for key in objects[0].keys():
                values = [obj[key] for obj in objects]
                zipped[key] = deep_zip(values, depth-1, passable_types=passable_types)
            return zipped
    return tuple(objects)

In [28]:
objects = [
    [(1, 2), ([3, 9], 4), {'a': 4}],
    [(5, 6), ([7, 10], 8), {'a': 5}],
]
depth = 10
print(deep_zip(objects, depth, passable_types=(list, tuple)))

[((1, 5), (2, 6)), ([(3, 7), (9, 10)], (4, 8)), ({'a': 4}, {'a': 5})]


In [22]:
def apply_func_on_depth(obj, func, depth, passable_types=(list, tuple, dict)):
    if depth != 0 and isinstance(obj, passable_types):
        if isinstance(obj, (list, tuple)):
            processed = list()
            for elem in obj:
                processed.append(apply_func_on_depth(elem, func, depth-1, passable_types=passable_types))
            if isinstance(obj, tuple):
                processed = tuple(processed)
            return processed
        elif isinstance(obj, dict):
            processed = dict()
            for key, value in obj.items():
                processed[key] = apply_func_on_depth(value, func, depth-1, passable_types=passable_types)
            return processed
    return func(obj)

In [27]:
obj = [
    [(1, 2), ([3, 9], 4), {'a': 4}],
    [(5, 6), ([7, 10], 8), {'a': 5}],
]

func = lambda x: x**2
depth = 4
print(apply_func_on_depth(obj, func, depth))

[[(1, 4), ([9, 81], 16), {'a': 16}], [(25, 36), ([49, 100], 64), {'a': 25}]]


In [1]:
import tensorflow as tf
a = tf.Variable(0)
ndim_tensor = tf.shape(tf.shape(a))
is_scalar = tf.equal(tf.shape(tf.shape(a))[0], 0)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(is_scalar))

True


In [2]:
f = lambda: 1
print(f())

1


In [6]:
a = {'a': 1, 'b': 2, 'c': 3}
def f(*args):
    print(args)

f(*a.items())

(('a', 1), ('b', 2), ('c', 3))


In [3]:
def f(a=1):
    print(a)
    
kwargs = {'a': 2}

f(4, **kwargs)

TypeError: f() got multiple values for argument 'a'

In [7]:
d = {([1, 2], [3, 4]): 1, ([5, 6], [7, 8]): 2}

TypeError: unhashable type: 'list'

In [13]:
print(not 0)

True


In [14]:
a = [1, 2]
print(a['1'])

TypeError: list indices must be integers or slices, not str

In [9]:
from deeppavlov.core.data.simple_vocab import SimpleVocabulary

# no exception
sv = SimpleVocabulary(
    load_path="file_which_does_not_exists",
    save_path="file_which_does_not_exists"
)
print(sv.load_path.is_file())
sv.load()
print(len(sv))

print()
# exception
sv = SimpleVocabulary(
    load_path="no_such_dir/file_which_does_not_exists",
    save_path="no_such_dir/file_which_does_not_exists"
)
print(sv.load_path.parent.is_dir())
print(sv.load_path.is_file())
sv.load()
print(len(sv))

False
0

True
False
0


In [3]:
import os
try:
    user_paths = os.environ['PYTHONPATH'].split(os.pathsep)
except KeyError:
    user_paths = []
print(user_paths)

[]


In [11]:
from collections import Counter
from itertools import chain
l = [1, 2, 3, 2, 1, 1]
freqs = Counter(chain(l))
print(list(freqs.most_common()))

[(1, 3), (2, 2), (3, 1)]


In [5]:
import re
s0 = 'abc\ndef'
s1 = repr(s0)
print(s1)
s2 = bytes(s, "utf-8").decode("unicode_escape")
print(s2)

'abc\ndef'
abc
def


In [7]:
def f(a: int) -> int:
    print(a)
    return [3]
    
f([1, 2])

[1, 2]


[3]

In [18]:
import numpy as np
vocab = ['a', 'b', 'c']
a = np.random.rand(10, 3)
def f(vec):
    return vocab[np.argmax(vec)]
b = np.apply_along_axis(f, -1, a)
print(''.join(b))
print(b.dtype.name)
print(np.dtype(str).name)
print(np.dtype(np.array(['a', 'b'])))

ccaccbaaba
str32
str


TypeError: data type not understood

In [23]:
import numpy as np
a = np.array([['a', 'b']])
print(a.dtype.name)

str32


In [6]:
from tensorflow.nn.rnn_cell import LSTMStateTuple as LSTMStateTuple
t1 = LSTMStateTuple(h='b', c='a')
t2 = LSTMStateTuple(c='a', h='b')
print(isinstance(t1, tuple))
for e in t1:
    print(e)
for e in zip(t1, t2):
    print(e)

True
a
b
('a', 'a')
('b', 'b')


In [38]:
import tensorflow as tf


def is_scalar_tensor(t):
    return tf.equal(tf.shape(tf.shape(t))[0], 0)


def replace_empty_saved_state(saved_and_zero_state):
    return tf.cond(
        is_scalar_tensor(saved_and_zero_state[0]),
        true_fn=lambda: saved_and_zero_state[1],
        false_fn=lambda: tf.reshape(saved_and_zero_state[0], tf.shape(saved_and_zero_state[1])),
    )

def synchronous_flatten(*nested):
    if not isinstance(nested[0], (tuple, list, dict)):
        return [[n] for n in nested]
    output = [list() for _ in nested]
    if isinstance(nested[0], dict):
        for k in nested[0].keys():
            flattened = synchronous_flatten(*[n[k] for n in nested])
            for o, f in zip(output, flattened):
                o.extend(f)
    else:
        try:
            for inner_nested in zip(*nested):
                flattened = synchronous_flatten(*inner_nested)
                for o, f in zip(output, flattened):
                    o.extend(f)
        except TypeError:
            print('(synchronous_flatten)nested:', nested)
            raise
    return output

def compose_save_list(*pairs, name_scope='save_list'):
    with tf.name_scope(name_scope):
        save_list = list()
        for pair in pairs:
            # print('pair:', pair)
            [variables, new_values] = synchronous_flatten(pair[0], pair[1])
            # print("(useful_functions.compose_save_list)variables:", variables)
            # variables = flatten(pair[0])
            # # print(variables)
            # new_values = flatten(pair[1])
            for variable, value in zip(variables, new_values):
                save_list.append(tf.assign(variable, value, validate_shape=False))
        return save_list

# saved_state = (
#     tf.Variable(0., validate_shape=False, trainable=False, name='state'),
#     tf.Variable(0., validate_shape=False, trainable=False, name='state')
# )
saved_state = tf.Variable(0., validate_shape=False, trainable=False, name='state')

lstm = tf.nn.rnn_cell.LSTMCell(10, state_is_tuple=False)

inp = tf.zeros([100, 10, 13])
zero_state = lstm.zero_state(tf.shape(inp)[1], tf.float32)

# state = [
#     replace_empty_saved_state([s, z]) for s, z in zip(saved_state, zero_state)
# ]
state = replace_empty_saved_state([saved_state, zero_state])
output, new_state = tf.nn.dynamic_rnn(
    lstm, inp, initial_state=state, parallel_iterations=1024, time_major=True
)


opt = tf.train.GradientDescentOptimizer(1.)

save_list = compose_save_list((saved_state, new_state))
with tf.control_dependencies(save_list):
    loss = tf.reduce_sum(output)
    grads_and_vars = opt.compute_gradients(loss, var_list=tf.global_variables())
    grads_and_vars = [(grad, var) for grad, var in grads_and_vars if grad is not None]
    train_op = opt.apply_gradients(grads_and_vars)
    
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(100):
        sess.run(train_op)
        print(i)

2018-09-13 15:26:54.204 WARNING in 'tensorflow'['tf_logging'] at line 120: <tensorflow.python.ops.rnn_cell_impl.LSTMCell object at 0x7f8bfc62b940>: Using a concatenated state is slower and will soon be deprecated.  Use state_is_tuple=True.


InvalidArgumentError: var and delta do not have the same shape[10,20] []
	 [[Node: GradientDescent/update_state/ApplyGradientDescent = ApplyGradientDescent[T=DT_FLOAT, use_locking=false, _device="/job:localhost/replica:0/task:0/device:GPU:0"](state, GradientDescent/learning_rate, gradients/cond/Reshape/Switch_grad/cond_grad)]]

Caused by op 'GradientDescent/update_state/ApplyGradientDescent', defined at:
  File "/usr/local/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/anton/dpenv/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/anton/dpenv/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/anton/dpenv/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 497, in start
    self.io_loop.start()
  File "/h

In [20]:
import numpy as np
indices_batch = np.array([None, None])
print(indices_batch[0] is None)
a = np.array([1, 2])
for i in np.nditer(indices_batch, flags=['refs_ok']):
    print(i.dtype.name)
for i in np.nditer(a, flags=['refs_ok']):
    print(i.dtype.name)
if any([i is None for i in np.nditer(indices_batch, flags=['refs_ok'])]):
    print('!')
    g = np.vectorize(lambda x: x is None)
    mask = g(indices_batch)
    print(mask)

True
object
object
int64
int64


In [21]:
print(repr('\n'))

'\n'


In [32]:
import numpy as np
a = np.array([[[1, 4], [2, 5], [3, 6]], [[7, 8], [9, 10], [11, 12]]])
b = np.reshape(a, (-1, a.shape[-1]))
correct = sum([y1 == y2 for y1, y2 in zip(a, a)])
for y1, y2 in zip(a, a):
    print(y1, y2, '\n')
print(correct, '\n')
print(len(a))

[[1 4]
 [2 5]
 [3 6]] [[1 4]
 [2 5]
 [3 6]] 

[[ 7  8]
 [ 9 10]
 [11 12]] [[ 7  8]
 [ 9 10]
 [11 12]] 

[[2 2]
 [2 2]
 [2 2]] 

2


In [35]:
import sys
import numpy as np
from deeppavlov.core.common.log import get_logger


def prepare_for_accuracy_computation(y_true, y_predicted):
    log = get_logger(__name__)
    if not isinstance(y_true, np.ndarray):
        y_true = np.array(y_true)
    if not isinstance(y_predicted, np.ndarray):
        y_predicted = np.array(y_predicted)
    num_examples = y_true.size
    if y_true.size != y_predicted.size:
        num_examples = min(y_true.size, y_predicted.size)
        log.warning(
            "Number of elements in y_true and in y_predicted are not equal\n"
            "y_true.size = {}, y_predicted.size = {}\n"
            "Using first {} elements of y_true and y_predicted".format(
                y_true.size, y_predicted.size, num_examples,
            )
        )
        y_true, y_predicted = np.reshape(y_true, (-1)), np.reshape(y_predicted, (-1))
        y_true, y_predicted = y_true[:num_examples], y_predicted[:num_examples]
    if y_true.shape != y_predicted.shape:
        log.warning(
            "y_true and y_predicted have different shapes\ny_true.shape = {}, y_predicted.shape = {}\n"
            "Reshaping y_true to y_predicted.shape".format(y_true.shape, y_predicted.shape)
        )
        y_true = np.reshape(y_true, y_predicted.shape)
    return y_true, y_predicted, num_examples


def accuracy(y_true, y_predicted):
    """
    Calculate accuracy in terms of absolute coincidence

    Args:
        y_true: array of true values
        y_predicted: array of predicted values

    Returns:
        portion of absolutely coincidental samples
    """
    
    y_true, y_predicted, num_examples = prepare_for_accuracy_computation(y_true, y_predicted)
    correct = np.sum(y_true == y_predicted)
    return correct / num_examples if num_examples else 0

In [37]:
y_true = np.array([[1, 2, 4]])
y_predicted = []
print(accuracy(y_true, y_predicted))

2018-09-13 14:25:53.122 WARNING in '__main__'['<ipython-input-35-9e32c49161a5>'] at line 22: Number of elements in y_true and in y_predicted are not equal
y_true.size = 3, y_predicted.size = 0
Using first 0 elements of y_true and y_predicted


True
0


In [32]:
print(np.mean(np.array([])))

nan


/home/anton/dpenv/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/anton/dpenv/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [40]:
a = np.array([1, 2, 3])
l = []
l += list(a)
print(l)

[1, 2, 3]


In [1]:
import numpy as np
a = np.array(['a', 'b'])
for e in np.nditer(a):
    print(type(e))
    print(str(e))

<class 'numpy.ndarray'>
a
<class 'numpy.ndarray'>
b


In [4]:
def f():
    global tf
    import tensorflow as tf
f()   
print(tf.__version__)

1.10.1


In [3]:
from tensorflow.contrib.cudnn_rnn import CudnnLSTM as CudnnLSTM
import tensorflow as tf

lstm = CudnnLSTM(1, 100)
inp = tf.placeholder(shape=[None, None, 31], dtype=tf.float32)
output, state = lstm(inp)
print(output)

Tensor("cudnn_lstm_1/CudnnRNN:0", shape=(?, ?, 100), dtype=float32)


In [3]:
import tensorflow as tf
a = tf.zeros([0, 100])
b = tf.ones([100, 50])
d = tf.split(b, [100, 0], axis=0)
print(d)
c = tf.matmul(a, b)
print(c)
with tf.Session() as sess:
    sess.run(c)

[<tf.Tensor 'split:0' shape=(100, 50) dtype=float32>, <tf.Tensor 'split:1' shape=(0, 50) dtype=float32>]
Tensor("MatMul_2:0", shape=(0, 50), dtype=float32)


In [5]:
import math
import turtle
turtle.shape('turtle')
a = 50
for i in range(360):
    t = i * math.pi/180
    x = 2*a*math.cos(t) - a*math.cos(2*t) - a
    y = 2*a*math.sin(t) - a*math.sin(2*t)
    turtle.goto(x, y)

In [2]:
import tensorflow as tf

a = tf.zeros([100, 100, 100])
b = tf.reduce_mean(a, axis=tf.constant([1, 2]))
c = tf.reduce_mean(a, axis=4)
with tf.Session() as sess:
    sess.run(c)

ValueError: Invalid reduction dimension 4 for input with 3 dimensions. for 'Mean_2' (op: 'Mean') with input shapes: [100,100,100], [] and with computed input tensors: input[1] = <4>.

In [1]:

import turtle
turtle.shape('turtle')
turtle.forward(50)
turtle.left(90)
turtle.forward(50)
turtle.left(90)
turtle.forward(50)
turtle.right(90)
turtle.forward(50)
turtle.right(90)
turtle.forward(50)
turtle.right(90)
turtle.forward(50)
turtle.right(90)
turtle.forward(50)
turtle.left(90)
turtle.forward(50)
turtle.left(90)
turtle.forward(50)

In [1]:
import turtle
turtle.shape('turtle')
turtle.forward(50)
turtle.left(90)
turtle.forward(50)
turtle.left(90)
turtle.forward(50)
turtle.right(90)
turtle.forward(50)
turtle.right(90)
turtle.forward(50)
turtle.right(90)
turtle.forward(50)
turtle.right(90)
turtle.forward(50)
turtle.left(90)
turtle.forward(50)


In [1]:
import math
import turtle
x = 0
y = 0
a = 100
turtle.reset
turtle.down
for i in range(0, 360):
    x = 2*a*math.cos(math.radians(i))-a*math.cos(2*math.radians(i))
    y = 2*a*math.sin(math.radians(i))-a*math.sin(2*math.radians(i))
    turtle.goto(x,y)

In [2]:
import math
import turtle
x = 0
y = 0
a = 100
turtle.reset
turtle.down
for i in range(0, 360):
    x = 2*a*math.cos(math.radians(i))-a*math.cos(2*math.radians(i))
    y = 2*a*math.sin(math.radians(i))-a*math.sin(2*math.radians(i))
    turtle.goto(x,y)

In [ ]:

f=True

while f:
    n=int(input('введите число 0=exit : '))
    if n==0: break
    a= [i for i in range(n+1)]
    b=[]
    print (a)

    for i in range(2,n):
        if (i**2<=n) and (a[i]!=0):
            for j in range(i**2,n+1,i):
                a[j]=0
            #print(a)    
    for i in range(n+1):
        if a[i]!=0:
            b.append(a[i])              
    print(b)

введите число 0=exit : 100
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]
[1, 2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]


In [1]:
import tensorflow as tf
import numpy as np
with tf.device('/gpu:0'):
    a = tf.placeholder(tf.int32)
    b = tf.argmax(a, axis=-1)
with tf.Session() as sess:
    sess.run(b, feed_dict={a: np.zeros(shape=(10, 20, 100), dtype=np.int32)})

InvalidArgumentError: Cannot assign a device for operation 'ArgMax': Could not satisfy explicit device specification '/device:GPU:0' because no supported kernel for GPU devices is available.
Registered kernels:
  device='GPU'; T in [DT_DOUBLE]; output_type in [DT_INT32]; Tidx in [DT_INT32]
  device='GPU'; T in [DT_DOUBLE]; output_type in [DT_INT64]; Tidx in [DT_INT32]
  device='GPU'; T in [DT_FLOAT]; output_type in [DT_INT32]; Tidx in [DT_INT32]
  device='GPU'; T in [DT_FLOAT]; output_type in [DT_INT64]; Tidx in [DT_INT32]
  device='GPU'; T in [DT_HALF]; output_type in [DT_INT32]; Tidx in [DT_INT32]
  device='GPU'; T in [DT_HALF]; output_type in [DT_INT64]; Tidx in [DT_INT32]
  device='CPU'; T in [DT_DOUBLE]; output_type in [DT_INT32]
  device='CPU'; T in [DT_DOUBLE]; output_type in [DT_INT64]
  device='CPU'; T in [DT_FLOAT]; output_type in [DT_INT32]
  device='CPU'; T in [DT_FLOAT]; output_type in [DT_INT64]
  device='CPU'; T in [DT_BFLOAT16]; output_type in [DT_INT32]
  device='CPU';

In [4]:
d = {k: list() for k in {'a': 1, 'b': 2}.keys()}
print(d)

{'a': [], 'b': []}


In [2]:
import tensorflow as tf

a=tf.Variable(tf.constant([0.,1.,2.]))
b=tf.Variable(tf.constant([1.,1.,1.]))
recall=tf.metrics.recall(b,a)

init_g = tf.global_variables_initializer()
init_l = tf.local_variables_initializer()
with tf.Session() as sess:
    sess.run([init_g])
    rec=sess.run(recall)
    print(rec)

FailedPreconditionError: Attempting to use uninitialized value recall_1/true_positives/count
	 [[Node: recall_1/true_positives/count/read = Identity[T=DT_FLOAT, _class=["loc:@recall_1/true_positives/AssignAdd"], _device="/job:localhost/replica:0/task:0/device:GPU:0"](recall_1/true_positives/count)]]
	 [[Node: recall_1/value/_3 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_58_recall_1/value", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'recall_1/true_positives/count/read', defined at:
  File "/usr/local/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new

In [5]:
d = {'a': 1}
print(hasattr(d, '__setitem__'))
l = []
print(hasattr(l, 'append'))
print(hasattr(l, '__getitem__'))
print(hasattr(l, '__setitem__'))

True
True
True
True


In [6]:
from collections import OrderedDict
d = OrderedDict([('a', 1)])
print(isinstance(d, dict))

True


In [1]:
import os
import tensorflow as tf
from tensorflow.contrib.cudnn_rnn import CudnnLSTM as CudnnLSTM
inp = tf.zeros([10, 32, 100])
lstm1 = CudnnLSTM(1, 128)
lstm2 = CudnnLSTM(1, 256)
_, _ = lstm1(inp)
_, _ = lstm2(inp)
saver = tf.train.Saver(
    [lstm1.saveable, lstm2.saveable]
)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    save_path = 'test_cudnn_lstm_save'
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    saver.save(sess, save_path)

ValueError: Parent directory of test_cudnn_lstm_save doesn't exist, can't save.

In [1]:
print([1, 2] == (1, 2))

False


In [3]:
print(int(6.4e6))

6400000


In [1]:
import helmo.experiments.test


/home/anton/h-elmo/helmo/experiments/test.py
/home/anton/h-elmo/abc/def


In [2]:
l = [1, 2]
print(l and True)

True


In [2]:
import os
print(os.path.split('/'))
print(os.path.split('/home'))
print(os.path.join('/', 'home'))

('/', '')
('/', 'home')
/home


In [3]:
import os
def full_path_split(path):
    splitted_path = list()
    head, tail = os.path.split(path)
    while len(head) > 0 and len(tail) > 0:
        splitted_path = [tail] + splitted_path
        head, tail = os.path.split(head)
    splitted_path = [head] + splitted_path if len(head) > 0 else [tail] + splitted_path
    return splitted_path

print(full_path_split('/home/peganov'))
print(full_path_split('home/peganov'))
print(full_path_split('~/peganov'))

['/', 'home', 'peganov']
['home', 'peganov']
['~', 'peganov']


In [4]:
a = False
b = ''
print(a is False)
print(b is False)

True
False


In [5]:
s = {'c', 'a', 'b'}
print(sorted(s))

['a', 'b', 'c']


In [1]:
import tensorflow as tf
from tensorflow.contrib.cudnn_rnn import CudnnLSTM as CudnnLSTM
lstm = CudnnLSTM(1, 100)
print(lstm.name)

cudnn_lstm_1


In [1]:

def Determinant(M):                                                 # список из списков-строк
    if len(M[0]) == 1:
        return(M[0])
    elif len(M[0]) == 2:
        return M[0][0] * M[1][1] - M[0][1]*M[1][0]    
    else:                                                          # раскладываем по первой строке
        
        MatrixLength = len(M)
        det = 0
        for ind, el in enumerate(M[0]):                            # итерируемся по минорам
            Minor = []
            for i in range (1, MatrixLength):                       # итерируемся по строкам
                MatrixString = []
                for j in range(MatrixLength):
                    if not (j == ind):
                        MatrixString.append(M[i][j])  
                Minor.append(MatrixString) 
            det += (-1)**ind * el * Determinant(Minor)
        return (det)
            
print(Determinant([[1]]))

[1]


In [2]:
print(Determinant([[2, 2, 2], [1, 1, 1], [1, 2, 3]]))

0


In [3]:
def Determinant(M):                                                 # список из списков-строк
    if len(M[0]) == 1:
        return(M[0])
    elif len(M[0]) == 2:
        return M[0][0] * M[1][1] - M[0][1]*M[1][0]    
    else:                                                          # раскладываем по первой строке
        
        num_rows = len(M)
        det = 0
        for c_i, el in enumerate(M[0]):                            # итерируемся по минорам
            submatrix = []
            for r_i in range (1, num_rows):                       # итерируемся по строкам
                submatrix.append(M[r_i][:c_i] + M[r_i][c_i+1:])
            det += (-1)**c_i * el * Determinant(submatrix)
        return (det)
    
print(Determinant([[2, 2, 2], [1, 1, 1], [1, 2, 3]]))

0


In [6]:
a = [int(e) for e in input().split()]
b = []
for i in range(len(a)-1):
    if a[i] > 2 or (a[i] == 2 and a[i+1] == 2):
        print('a[i] ', a[i])
        b.append(a[i])
       
print(a,len(a))
print((sum(b)+a[len(a)-1])//(len(b)+1))

5 5 5 5 5 5 5 5 5 2
a[i]  5
a[i]  5
a[i]  5
a[i]  5
a[i]  5
a[i]  5
a[i]  5
a[i]  5
a[i]  5
[5, 5, 5, 5, 5, 5, 5, 5, 5, 2] 10
4
